In [ ]:
import numpy as np 
import pandas as pd 


In [ ]:
import json

In [ ]:
pip install response

In [ ]:
import response

In [ ]:
import requests

## Converting the API request for AE into Dataframe

In [ ]:
ae_data= requests.get("https://pyhack-dot-pharmanlp-177020.uc.r.appspot.com/api/1/StudyHack/ae/subject/105792/list")

In [ ]:
print(ae_data.status_code)

In [ ]:
print(ae_data.json())


In [ ]:
ae_df = pd.json_normalize(ae_data.json(), record_path=['data'],meta =['status','request_id'])

In [ ]:
ae_df.head()

# Converting the API request for CM subject list into Dataframe

In [ ]:
cm_subj_data= requests.get("https://pyhack-dot-pharmanlp-177020.uc.r.appspot.com/api/1/StudyHack/ae/subject/105792/list")

In [ ]:
print(cm_subj_data.status_code)

In [ ]:
print(cm_subj_data.json())

In [ ]:
cm_subj_df = pd.json_normalize(cm_subj_data.json(), record_path=['data'],meta =['status','request_id'])

In [ ]:
cm_subj_df['subjectid'].value_counts()

# Converting the API request for AE subject list into Dataframe

In [ ]:
ae_subj_data= requests.get("https://pyhack-dot-pharmanlp-177020.uc.r.appspot.com/api/1/StudyHack/ae/subject/list")

In [ ]:
print(ae_subj_data.json())

In [ ]:
ae_subj_df = pd.json_normalize(ae_subj_data.json(), record_path=['data'],meta =['status','request_id'])

In [ ]:
ae_subj_df.head()

In [ ]:
ae_subj_df['status'].value_counts()

###  In the above API's, got the data from both AE api. For CM, subject list api has the required details and using same AE subject ids' for CM data

### Fetching the AE data by looping subject ids'

In [ ]:
ae_ls = ae_subj_df.loc[:,0].tolist()[0:10]

In [ ]:
len(ae_ls)

In [ ]:
df = []
for i in ae_ls: 
    if i is not None: 
        temp =requests.get("https://pyhack-dot-pharmanlp-177020.uc.r.appspot.com/api/1/StudyHack/ae/subject/"+str(i)+"/list")
        df.append(temp.json())
    else:
        continue
        
        

In [ ]:
df

In [ ]:
final_ae_df = pd.json_normalize(df, record_path=['data'],meta =['status','request_id'])

In [ ]:
final_ae_df['subjectid'].value_counts()

In [ ]:
len(final_ae_df)

### Fetching the CM data by looping subject ids'

In [ ]:
cm_df = []
for i in ae_ls: 
    if i is not None: 
        temp =requests.get("https://pyhack-dot-pharmanlp-177020.uc.r.appspot.com/api/1/StudyHack/cm/subject/"+str(i)+"/list")
        cm_df.append(temp.json())
    else:
        continue

In [ ]:
final_cm_df = pd.json_normalize(cm_df, record_path=['data'],meta =['status','request_id'])

In [ ]:
final_cm_df['subjectid'].value_counts()

In [ ]:
len(final_cm_df)

#### Converting object format into float - final_cm_df['cmaeno']

In [ ]:
final_ae_df['aespid'].dtypes

In [ ]:
final_cm_df['cmaeno'].dtypes

In [ ]:
final_cm_df['cmaeno'] = pd.to_numeric(final_cm_df['cmaeno'],errors= 'coerce').fillna(0,downcast= 'infer')

In [ ]:
final_ae_df = final_ae_df.drop_duplicates()

In [ ]:
final_cm_df = final_cm_df.drop_duplicates()

### Combining both AE and CM to find type errors

In [ ]:
final_df = pd.merge(final_cm_df, final_ae_df
                    
        
                    
                    ,how = 'inner' ,right_on = ['subjectid','aespid'], left_on = ['subjectid','cmaeno'] )

In [ ]:
Removed the Adverse events who never gets into medication using the above join functions

In [ ]:
pip install pandasql

In [ ]:
from pandasql import sqldf 

In [ ]:
final_df.to_csv("final_df.csv")

In [ ]:
# Checking Medications are given and Adverse Event occur don't match

In [ ]:
check1 = sqldf(select   from final_df where subjectid)